 # 웹크롤링과 텍스트데이터 시각화

### Outline

### 6. Post 방식을 이용한 웹크롤링
    6.1 정적 웹페이지의 웹크롤링
    6.2 동적 웹페이지의 웹크롤링

#### 6.1 정적 웹페이지의 웹크롤링

- 필요한 모듈 및 라이브러리 설치
- 웹크롤링을 하려고 하는 웹사이트의 URL 및 데이터 지정

In [ ]:
import re
import requests as req
from bs4 import BeautifulSoup as bs
from IPython.display import Image

In [ ]:
url = 'http://kiss.kstudy.com/search/sch-result.asp'
data = {'json': '{"prefixQuery":{"inst":null,"publ":null,"exYear":null,"issn":null,"mbcdls":[],"publPldv":null,"ciInfo":{"title":"등재정보","field":"CI_INFO","value":[{"title":"KCI등재","value":"KCI등재"},{"title":"SCOPUS","value":"SCOPUS"}]}},"collectionQuery":{"queries":[{"field":"전체","value":"빅데이터","prefix":""}],"reQueries":[]},"dateRange":null,"sort":null,"isForwardMatch":false,"isContainsAttach":false}',
        'startCount': '0',  # 첫 번째 페이지
        'pageScale': '100'} # 한 페이지에 100개의 논문 리스트 나열

(1) Post 방식으로 서버에 정보 요청 및 응답 받음

In [ ]:
res  = req.post(url = url,data = data)

(2) HTML 파싱

In [ ]:
soup = bs(res.text,'html.parser')

(3) 원하는 정보 (키워드 리스트) 추출

In [ ]:
Image(filename='KISS01.png', width=700)

In [ ]:
kws  = soup.select('div.key-words')
kws[0]

(4) 추출한 정보의 전처리

- a태그의 수준에서 키워드를 이중리스트로 묶음

In [ ]:
kws = [kw.select('span > a') for kw in kws]
kws[0]

- 텍스트만 분리하고, 공백 제거

In [ ]:
kws = [[x.text.strip() for x in kw] for kw in kws]
kws[:3]

- 한글만 추출

In [ ]:
kws = [[re.sub("[^가-힣]","",x) for x in kw] for kw in kws]
kws[:3]

- 공백 제거

In [ ]:
kws = [[x for x in kw if len(x)>0] for kw in kws]
kws[:5]

#### 6.2 동적 웹페이지의 웹크롤링

- 페이지 수가 넘어감에 따라 새로운 정보를 요청하고 응답 받음

In [ ]:
Image(filename='KISS02.png', width=700)

In [ ]:
import time # 시간 관련 함수
import math # 수학 관련 함수

- 검색어를 지정하면, 자동으로 모든 학술논문에서 키워드를 추출하는 함수 생성

In [ ]:
Image(filename='KISS03.png', width=700)

In [ ]:
def keyword_crawler(searchword):

    # URL 및 데이터 지정
    url  = 'http://kiss.kstudy.com/search/sch-result.asp'
    data = {'json': '{"prefixQuery":{"inst":null,"publ":null,"exYear":null,"issn":null,"mbcdls":[],"publPldv":null,"ciInfo":{"title":"등재정보","field":"CI_INFO","value":[{"title":"KCI등재","value":"KCI등재"},{"title":"SCOPUS","value":"SCOPUS"}]}},"collectionQuery":{"queries":[{"field":"전체","value":"'+searchword+'","prefix":""}],"reQueries":[]},"dateRange":null,"sort":null,"isForwardMatch":false,"isContainsAttach":false}',
            'startCount': '0',
            'pageScale': '10'}

    # Post 방식으로 서버에 정보 요청 및 정보 받음
    res = req.post(url = url,data = data)

    # HTML 파싱
    soup = bs(res.text,'html.parser')

    # 원하는 정보 (전체 논문 수) 추출
    N = soup.select('div.list-title.is1 > span.count > span.fc-key')[0].text
    N = re.sub(',','',N)
    N = int(N)
    N = math.ceil(N/100) # 올림

    time.sleep(1) # 시간 1초 지연

    output = []
    for i in range(N):

        # 시작 페이지 번호를 변경할 수 있도록 수정
        data = {'json': '{"prefixQuery":{"inst":null,"publ":null,"exYear":null,"issn":null,"mbcdls":[],"publPldv":null,"ciInfo":{"title":"등재정보","field":"CI_INFO","value":[{"title":"KCI등재","value":"KCI등재"},{"title":"SCOPUS","value":"SCOPUS"}]}},"collectionQuery":{"queries":[{"field":"전체","value":"'+searchword+'","prefix":""}],"reQueries":[]},"dateRange":null,"sort":null,"isForwardMatch":false,"isContainsAttach":false}',
                'startCount': str(i*100),
                'pageScale': '100'}

        # Post 방식으로 서버에 정보 요청 및 정보 받음
        res = req.post(url = url,data = data)

        # HTML 파싱
        soup = bs(res.text,'html.parser')

        # 원하는 정보 (키워드) 추출
        kws = soup.select('div.key-words')

        kws = [kw.select('span > a') for kw in kws]
        kws = [[x.text.strip() for x in kw] for kw in kws]
        kws = [[re.sub("[^가-힣]","",x) for x in kw] for kw in kws]
        kws = [[x for x in kw if len(x)>0] for kw in kws]

        output += kws

        time.sleep(1) # 시간 1초 지연

    return output

In [ ]:
kws = keyword_crawler('빅데이터')

In [ ]:
import pandas as pd
kws = pd.DataFrame(kws)
kws.shape

In [ ]:
kws.to_csv('Keywords.csv')